<a href="https://colab.research.google.com/github/Dhruv-Ganapati/Emotion_Detection_usingAI/blob/main/Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Header: Emotion Detection:

### Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/AI-Udemy/Bonus Projects/Affectiva AI

/content/drive/MyDrive/Colab Notebooks/AI-Udemy/Bonus Projects/Affectiva AI


In [ ]:
import pandas as pd
import numpy as np
import os
import PIL
import seaborn as sns
import pickle
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.keras import *
from tensorflow.python.keras import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from keras import optimizers

from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df

In [ ]:
df.head(5)

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.describe().T

Here, 30 columns are Floating type data set and Remaining 1 data set is image(Object type).

Images contains values of it's color.
* 0 : represent full balck object
* 255: represent white image object
* Between 0-255: represent Gray-scale image.

In [ ]:
df['Image']

In [ ]:
df['Image'].shape

In [ ]:
df['Image'].size

In [ ]:
# Since values for the image are given as space separated string, separate the values using ' ' as separator.
# Then convert this into numpy array using np.fromstring and convert the obtained 1D array into 2D array of shape (96, 96)
df['Image'] = df['Image'].apply(lambda x: np.fromstring(x, dtype = int, sep=' ').reshape(96,96))

In [ ]:
df['Image'][0].shape

MINI CHALLENGE #1:
- Obtain the average, minimum and maximum values for 'right_eye_center_x'

In [ ]:
df['right_eye_center_x'].describe()

### ==========================================================

# PERFORM IMAGE VISUALIZATION

In [ ]:
# Plot a random image from the dataset along with facial keypoints.
# Image data is obtained from df['Image'] and plotted using plt.imshow
# 15 x and y coordinates for the corresponding image
# since x-coordinates are in even columns like 0,2,4,.. and y-coordinates are in odd columns like 1,3,5,..
# we access their value using .loc command, which get the values for coordinates of the image based on the column it is refering to.

np.random.seed(78)    # For same image, seed helps to provide same  random value everytime
i = np.random.randint(1, len(df))
plt.imshow(df['Image'][i], cmap='gray')
for j in range(1,31,2):
  plt.plot(df.loc[i][j-1], df.loc[i][j], 'rx')

* <!-- i =[]
for j in range(1, 31, 2):
  i.append(j)
  print(i) -->

In [ ]:
#View more images in grid format
fig = plt.figure(figsize=(20,20),dpi=200)

i=[]
# for m in range(16):
#   i.append(np.random.randint(m,len(df)))    # Generate random images and stores on I[] list

# Generate 16 unique random numbers
while len(i) < 16:
    random_number = np.random.randint(0, len(df))  # Generate a random number
    if random_number not in i:   # Check if the number is unique
        i.append(random_number)  # Append unique random number
print("I: ", i)

for a in range(16):
  ax = fig.add_subplot(4,4,a+1)           # provide 16 empty image boxes
  a = i[a]                        # Putting random images loc on loop
  image = plt.imshow(df["Image"][a], cmap='gray')   # FIll all the related images on the boxes
  for j in range(1, 31, 2):
    plt.plot(df.loc[a][j-1], df.loc[a][j], 'rx')     # Fill the red 'x' facial point.


MINI CHALLENGE #2:
- Perform a sanity check on the data by randomly visualizing 64 new images along with their cooresponding key points

In [ ]:
import random

fig = plt.figure(figsize=(20,20))

x = []
while len(x) < 64:
  random_number = np.random.randint(0, len(df))
  if random_number not in x:
    x.append(random_number)

print(x)    # Generated 64 unique random number from data frame

for a in range(64):
  ax = fig.add_subplot(8,8, a+1)        # Here, 8,8 is 8 rows * 8 columns
  a = x[a]
  image = plt.imshow(df['Image'][a], cmap='gray')
  for j in range(1,31,2):
    plt.plot(df.loc[a][j-1], df.loc[a][j], 'rx')

# PERFORM IMAGE AUGMENTATION
Image augmentation is a technique commonly used in deep learning for computer vision tasks, such as image classification, object detection, and segmentation. It involves applying various transformations to the original images in the training dataset to create new training examples.
Some common image augmentation techniques include:

* Rotation: Rotating the image by a certain angle.
* Flip: Flipping the image horizontally or vertically.
* Zoom: Zooming in or out of the image.
* Translation: Shifting the image horizontally or vertically.
* Brightness adjustment: Changing the brightness level of the image.
* Contrast adjustment: Adjusting the contrast of the image.
* Noise injection: Adding random noise to the image.
* Crop and resize: Cropping a portion of the image and resizing it to the original size.
* Shear: Applying shearing transformations to the image.

In [ ]:
import copy
copy_df = copy.copy(df)

Generalization of model is very important than memorization. One can't memorise all the thing.

In [ ]:
# showing all the key facial point of the dataFrame
columns = copy_df.columns[:-1]
columns

###**Horizontal Flip**

In [ ]:
# Fliping image Horizontally
copy_df['Image'] = copy_df['Image'].apply(lambda x:np.flip(x, axis=1))

##since we are fliping Horizonatlly, y co-ordinates values would be the same.
# Only x-coordinates values would change, all we have to do is to substract our initial x-coordinates values form width of the image(96*96)
for i in range(len(columns)):
  if i%2 == 0:          # only even means: x-coordinates
    copy_df[columns[i]] = copy_df[columns[i]].apply(lambda x: 96.0 -float(x))      # 96 pixels -( x-cordinates)value


In [ ]:
df['left_eye_center_x']

In [ ]:
# Here value = 96 - df(value)
copy_df['left_eye_center_x']

In [ ]:
# Showing the original images as:
plt.imshow(df['Image'][0], cmap='gray')
for j in range(1, 31, 2):
  plt.plot(df.loc[0][j-1], df.loc[0][j], 'rx')

In [ ]:
# Showing the Flipped image = Horizontal flipped image
plt.imshow(copy_df['Image'][0], cmap='gray')
for j in range(1, 31, 2):
  plt.plot(copy_df.loc[0][j-1], copy_df.loc[0][j], 'rx')

**Augmented DataFrame** -concate(original + Horizontally flipped data)
* Note: The more we do augmentation, the data become more complicated.


Here we increase the brightness fo this image.

In [ ]:
  augmented_df = np.concatenate((df, copy_df))

In [ ]:
augmented_df.shape

**Brightness: Increase**
* np.clip: "Here CLIP helps to bring pixels values to the range between 0.0 to 255.0, if the pixel values goes out of range.
      * For eg: suppose x=200 pix
        then 200 * 2 = 400 pix which is out of range(0-255). Then np.CLIP bring it between range.

In [ ]:
# Randomly increasing the brightness of the image
# We multiply pixel values by random values range(1.5 -2.0) to incease the brightness
# We can clip the value between 0-255
import random

copy_df = copy.copy(df)

In [ ]:
copy_df['Image'] = copy_df['Image'].apply(lambda x:np.clip(random.uniform(1.5,2)*x, 0.0, 255.0))
augmented_df = np.concatenate((augmented_df, copy_df))
augmented_df.shape

In [ ]:
# show the image
plt.imshow(copy_df['Image'][0], cmap='gray')
for j in range(1,31,2):
  plt.plot(copy_df.loc[0][j-1], copy_df.loc[0][j], 'rx')


###**Vertical Flip**
Challenge MINI

In [ ]:
ver_df = copy.copy(df)

In [ ]:
cloumns = ver_df.columns[:-1]
columns.size

In [ ]:
#Vertical Flip [axis=0] - flip the image along x-axis
ver_df['Image'] = ver_df['Image'].apply(lambda x: np.flip(x, axis=0))

# Since we are flipping vertically, x-cordinates values would be same
# Only y-cordinates values would changes, all we to do is to substract our initial y-cordinates values from height:96
for i in range(len(columns)):
  if i % 2 == 1:
    ver_df[columns[i]] = ver_df[columns[i]].apply(lambda x: 96.0 -float(x))

In [ ]:
plt.imshow(ver_df['Image'][0], cmap='gray')
#show 'x' red points
for i in range(1, 31, 2):
  plt.plot(ver_df.loc[0][i-1], ver_df.loc[0][i], 'rx')

## PERFORMING DATA NORMALIZATION & taining data
* AUGMENTED Data Frame is in the form of NUMPY Array. So we can't extact images column by
* augmented_df['Image']

In [ ]:
# Obtain the value of images which is present in the 31st columns (Since Index start from 0, we refer to 31st column)
img = augmented_df[:, 30]
img

**Image Normalization:**
Image normalization is a common preprocessing step in computer vision tasks for several reasons:

1. **Better Model Convergence**: Normalizing pixel values to a fixed range, such as [0, 1] or [-1, 1], can help improve the convergence of optimization algorithms during training. This is because normalizing the input data reduces the scale of input features, which can lead to more stable training dynamics and faster convergence.

2. **Reduced Sensitivity to Scale**: Neural networks can be sensitive to the scale of input features. Normalizing pixel values to a fixed range helps mitigate this sensitivity, making the model more robust to changes in the scale of input data.

3. **Improved Generalization**: Normalizing images can help the model generalize better to unseen data by reducing the impact of variations in pixel intensity across different images. This can lead to better performance on validation and test sets.

4. **Avoiding Saturation**: Neural networks often use activation functions like sigmoid or tanh, which saturate at extreme values. By normalizing pixel values to a fixed range, you can avoid saturation of these activation functions and ensure that gradients flow effectively during training.

5. **Compatibility with Pretrained Models**: Many pretrained models and architectures, especially those trained on large-scale datasets like ImageNet, expect input images to be normalized in a specific way. By normalizing your input images to match the expectations of these models, you can effectively use transfer learning and leverage pretrained weights for your own tasks.

Overall, image normalization is a simple yet effective preprocessing technique that can improve the stability, convergence, and generalization of neural network models in computer vision tasks.

In [ ]:
# Normalize the images
img = img / 255       # dividing by highest pixel

# create an empty array of shape (x, 96, 96, 1) to feed the model
img

In [ ]:
# Create an empty array of shape(x, 96, 96, 1) to feed the model
X = np.empty((len(img), 96, 96, 1))

# Iterate through the img list and add image values to the empty aray after expanding it's dimension from (96, 96) to (96, 96, 1)
for i in range(len(img)):
  X[i, ] = np.expand_dims(img[i], axis=2)

# Convert the images to float32 to ensure division results in float values
X = np.array(X).astype(np.float32)

In [ ]:
X.shape

In [ ]:
# Obtain the values of x and y co-ordinates which are used as target
y = augmented_df[:, :30]      # excluding 31st column
y = np.asarray(y).astype(np.float32)
y.shape

In [ ]:
#X[0][0].shape

**TRAIN & TEST: SPLIT DATA**
* X_test and X_train: INput for testing and training data
* y_test and y_train: Output fo testing and taining data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

### BUILDING DEEP RESIDUAL NEURAL NETWORK KEY FACIAL POINTS DETECTION MODEL

![alt text](https://drive.google.com/uc?id=1q0If_tsIyz2GfOb9EsRH_8_ue-1QnTlu)

![alt text](https://drive.google.com/uc?id=1GDhehqRRtnTA3-i02cYcalbPA27ej7Ar)

**Creating RES-BLOCK**
* 1. CONVOLUTION BLOCK
* 2. IDENTITY BLOCK
* 3. IDENTITY BLOCK

Let's first create CONVOLUTION BLOCK

In [ ]:
def res_block(X, filter, stage):

  ##CONVOLUTION BLOCK
  X_copy = X
  f1, f2, f3 = filter

  #main path ---------------------------------------PART 1
  # 1. Conv2D
  # padding valid : default padding
  X= Conv2D(f1, kernel_size=(1,1), strides=(1,1), name='res_'+str(stage)+'_conv_a', kernel_initializer=glorot_uniform(seed=0))(X)

  #2. MaxPool2D
  X = MaxPool2D(pool_size=(2,2))(X)

  #3. BatchNorm, Relu
  X = BatchNormalization(axis=3, name="bn_"+str(stage)+"_conv_a")(X)
  X = Activation('relu')(X)

  #4. Conv2D - kernel(3*3)
  # Padding same : In this type of padding, we only append zero to the left of the array and to the top of the 2D input matrix.
  X = Conv2D(f2, kernel_size=(3,3), strides=(1,1), padding='same', name='res_'+str(stage)+'_conv_b', kernel_initializer=glorot_uniform(seed=0))(X)

  #5. BatchNorm, Relu
  X = BatchNormalization(axis=3, name='bn_'+str(stage)+'_conv_b')(X)
  X = Activation('relu')(X)

  #6. Conv2D
  X = Conv2D(f3, kernel_size=(1,1), strides=(1,1), name='res_'+str(stage)+'_conv_c', kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3, name='bn_'+str(stage)+'_conv_c')(X)



  #short path ---------------------------------------PART 2
  #1. Conv2D
  X_copy = Conv2D(f3, (1,1), (1,1), name='res_'+str(stage)+'_conv_copy', kernel_initializer=glorot_uniform(seed=0))(X_copy)

  #2. MaxPool2D
  X_copy = MaxPool2D(pool_size=(2,2))(X_copy)

  #3. BatchNorm
  X_copy = BatchNormalization(axis=3, name='bn_'+str(stage)+'_conv_copy')(X_copy)

  # ADDING BOTH PATH
  X = Add()([X, X_copy])
  X = Activation('relu')(X)



  ##IDENTITY BLOCK 1
  X_copy = X

  #main path
  #1. Conv2D
  X = Conv2D(f1, kernel_size=(1,1), strides=(1,1), name='res_'+str(stage)+'_identity_1_a', kernel_initializer=glorot_uniform(seed =0))(X)

  #2. BatchNorm, relu
  X = BatchNormalization(axis=3, name='bn_'+str(stage)+'_identity_1_a')(X)
  X = Activation('relu')(X)

  #3. Conv2D - kernel(3*3)
  X = Conv2D(f2, kernel_size=(3,3), strides=(1,1), padding='same', name='res_'+str(stage)+'_identity_1_b', kernel_initializer=glorot_uniform(seed=0))(X)

  #4. BatchNorm, relu
  X = BatchNormalization(axis=3, name='bn_'+str(stage)+'_identity_1_b')(X)
  X = Activation('relu')(X)

  #5. Conv2D
  X = Conv2D(f3, (1,1), (1,1), name='res_'+str(stage)+'_identity_1_c', kernel_initializer=glorot_uniform(seed=0))(X)

  #6. BatchNorm
  X = BatchNormalization(axis=3, name='bn_'+str(stage)+'_identity_1_c')(X)

  # Add
  X = Add()([X, X_copy])
  X = Activation('relu')(X)


  ##IDENTITY BLOCK 2
  X_copy = X

  #main path
  #1. Conv2D
  X = Conv2D(f1, kernel_size=(1,1), strides=(1,1), name='res_'+str(stage)+'_identity_2_a', kernel_initializer=glorot_uniform(seed=0))(X)
  #2. BatchNorm, relu
  X = BatchNormalization(axis=3, name='bn_'+str(stage)+'_identity_2_a')(X)
  X = Activation('relu')(X)

  #3. Conv2D -kernel(3*3)
  X = Conv2D(f2, (3,3), (1,1), padding='same', name='res_'+str(stage)+'_identity_2_b', kernel_initializer=glorot_uniform(seed=0))(X)
  #4. BatchNOrm, relu
  X = BatchNormalization(axis=3, name='bn_'+str(stage)+'_identity_2_b')(X)
  X = Activation('relu')(X)

  #5. Conv2D
  X = Conv2D(f3, (1,1), (1,1), name='res_'+str(stage)+'_identity_2_c', kernel_initializer=glorot_uniform(seed=0))(X)
  #6. BatchNorm
  X = BatchNormalization(axis = 3, name='bn_'+str(stage)+'_identity_2_c')(X)

  # Add
  X = Add()([X, X_copy])
  A = Activation('relu')(X)

  return X

![alt text](https://drive.google.com/uc?id=1G_gvfoKJyRGpq_oNejLwDj7ZxIZvCGk6)

### Creating FINAL MODEL

In [ ]:
from tensorflow.keras.layers import Input, ZeroPadding2D, Conv2D, BatchNormalization, Activation, MaxPooling2D, AveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Model


input_shape = (96,96,1)

#1. Input Tensor shape
X_input = Input(input_shape)

#2. ZeroPadding
X = ZeroPadding2D((3,3))(X_input)

#3. Conv2D, BatchNOrm, MaxPool2D
X = Conv2D(64, kernel_size=(7,7), strides=(2,2), name='conv1', kernel_initializer=glorot_uniform(seed = 0))(X)
X = BatchNormalization(axis=3, name= 'bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((3,3), strides=(2,2))(X)

#4. RES_BLOCK
X = res_block(X, filter=[64, 64, 256], stage=2)
X = res_block(X, filter= [128, 128, 512], stage=3)

#5. AveragePooling2D
X = AveragePooling2D((2,2), name='Average_Pooling')(X)

#6. Final Layer(Flatten(), [Dense Layer-Relu, Dropout]*2, Dense Layer-Relu)
X =Flatten()(X)
X = Dense(4096, activation = 'relu')(X)
X = Dropout(0.2)(X)

X = Dense(2048, activation='relu')(X)
X = Dropout(0.1)(X)

X = Dense(30, activation = 'relu')(X)



model_1_facialKeyPoints = Model(inputs = X_input, outputs=X)
model_1_facialKeyPoints.summary()

In [ ]:
model_1_facialKeyPoints.summary()

### COMPLIE & TRAIN KEY FACIAL POINTS DETECTION deep learning model

* adam = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    name='adam'
  )

In [ ]:
adam = tf.keras.optimizers.Adam(
    learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    name='adam'
  )
model_1_facialKeyPoints.compile(loss="mean_squared_error", optimizer=adam, metrics=['accuracy'])

In [ ]:
# using early stopping to exit training if validation loss is not decreasing even after certain epochs (patience)
earlystopping = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 20)

# save the best model with least validation loss
checkpointer = ModelCheckpoint(filepath='FacialKeyPoints_weight.h5', mode = 'min', verbose=1, save_best_only=True)


In [ ]:
history1 = model_1_facialKeyPoints.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_split = 0.01, callbacks=[checkpointer, earlystopping])

In [ ]:
# save the model architecture to json file for future use

model_json = model_1_facialKeyPoints.to_json()
with open("FacialKeyPoints-model.json", "w") as json_file:
  json_file.write(model_json)


## ASSESS TRANIED KEY FACIAL POINTS DETECTION MODEL PERFORMANCE

In [ ]:
from keras.models import model_from_json

# Load the model architecture from the JSON file
with open("FacialKeyPoints-model.json", "r") as json_file:
    loaded_model_json = json_file.read()

# Reconstruct the model from the loaded architecture (Loaded_model = model_1_facialKeyPoint ho)
model_1_facialKeyPoints = model_from_json(loaded_model_json)

# Load the trained weights
model_1_facialKeyPoints.load_weights("FacialKeyPoints_weight.h5")

adam = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
# Compile the loaded model (if needed)
model_1_facialKeyPoints.compile(optimizer=adam, loss='mse', metrics=['accuracy'])

# Use the loaded model for inference or further training
# For example:
# predictions = model_1_facialKeyPoint.predict(X_test)

In [ ]:
result = model_1_facialKeyPoints.evaluate(X_test, y_test)
print("Accuracy: {}".format(result[1]))

So, Here we got the accuracy of 74.143% on unseen testing data which is pretty good.
# Grid Search CV
### How to increase accuracy rate:
code :
from keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV

- Define the model
- Replace 'model' with your model architecture
model = ...

- Define the parameter grid
param_grid = {'lr': [0.001, 0.01, 0.1]}  # Learning rate values to try

- Define the optimizer
optimizer = Adam()

- Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

- Define GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

- Train the model
grid_search.fit(X_train, y_train)

- Get the best parameters
best_lr = grid_search.best_params_['lr']

- Use the best learning rate to train the model
optimizer = Adam(learning_rate=best_lr)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.01)

- Evaluate on test data
result = model.evaluate(X_test, y_test)
print("Accuracy: {}".format(result[1]))



In [ ]:
history1.history.keys()

In [ ]:
np.max(history1.history['accuracy'])

In [ ]:
# Create subplots with 1 rows and 2 column
fig, axes = plt.subplots(1, 2, figsize=(15, 6))


# Plot training & validation accuracy values
axes[0].plot(history1.history['accuracy'])
axes[0].plot(history1.history['val_accuracy'])
axes[0].set_title('Model accuracy')
axes[0].set_ylabel('Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].legend(['Train_Acc', 'Validation_Acc'], loc='upper left')
axes[0].grid(True)

# Subplot for loss
axes[1].plot(history1.history['loss'])
axes[1].plot(history1.history['val_loss'])
axes[1].set_title('Model loss')
axes[1].set_ylabel('Loss')
axes[1].set_xlabel('Epoch')
axes[1].legend(['Train_loss', 'Validation_loss'], loc='upper left')
axes[1].grid(True, color='gray')

# Adjust layout to prevent overlap
plt.tight_layout()

In [ ]:
#!pip install tensorflow scikit-learn

In [ ]:
# from sklearn.model_selection import GridSearchCV
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.optimizers import Adam


# # Define a function to create the Keras model
# def create_model(lr=0.01):
#     model = Sequential([
#         Dense(64, activation='relu', input_shape=(input_shape,)),  # Replace input_shape with your input shape
#         Dense(32, activation='relu'),
#         Dense(num_classes, activation='softmax')  # Replace num_classes with the number of output classes
#     ])
#     optimizer = Adam(learning_rate=lr)
#     model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
#     return model

# # Define the parameter grid
# param_grid = {'lr': [0.001, 0.01, 0.1]}  # Learning rate values to try

# # Create the Keras model
# keras_model = KerasClassifier(build_fn=create_model, verbose=0)

# # Define GridSearchCV
# grid_search = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=3)

# # Train the model
# grid_search.fit(X_train, y_train)

# # Get the best parameters
# best_lr = grid_search.best_params_['lr']

# # Use the best learning rate to train the model
# best_model = create_model(lr=best_lr)
# history = best_model.fit(X_train, y_train, epochs=4, batch_size=64, validation_split=0.01)

# # Evaluate on test data
# result = best_model.evaluate(X_test, y_test)
# print("Accuracy: {}".format(result[1]))


In [ ]:
#!pip install --upgrade --force-reinstall keras



**=================================================PART -2=======================================================**


### **FACIAL EXPRESSION(EMOTIOIN) DETECTION**

- DATA importing

In [ ]:
facial_df = pd.read_csv('icml_face_data.csv')

In [ ]:
facial_df.head()

In [ ]:
# @title null value
facial_df.isnull().sum()

In [ ]:
facial_df.describe()

In [ ]:
# @title emotion

from matplotlib import pyplot as plt
facial_df['emotion'].plot(kind='hist', bins=20, title='emotion')
plt.gca().spines[['top', 'right',]].set_visible(False)

**String -> array**
Everything all right. Let's convert string pixel into array format

In [ ]:
facial_df[' pixels']

In [ ]:
facial_df['emotion'].shape

In [ ]:
# function to convert pixel values in string format to array format
def string2array(x):
  return np.array(x.split(' ')).reshape(48,48,1).astype('float32')

In [ ]:
# function to resize images from (48, 48) to (96, 96)
def resize(x):
  img = x.reshape(48, 48)
  return cv2.resize(img, dsize=(96,96), interpolation=cv2.INTER_LINEAR)

In [ ]:
facial_df[' pixels'] = facial_df[' pixels'].apply(lambda x: string2array(x))

In [ ]:
facial_df[' pixels'] = facial_df[' pixels'].apply(lambda x: resize(x))

In [ ]:
facial_df.head(5)

In [ ]:
facial_df.columns

In [ ]:
# @title rename column
facial_df = facial_df.rename(columns={' pixels': 'pixels'})

In [ ]:
facial_df.columns

**Visualize the any image in the dataframe**

The image will be low pixels because we have converted it into (96,96) from low pixels value (48,48).

In [ ]:
plt.imshow(facial_df['pixels'].iloc[-1], cmap='gray')

### **VISUALIZE IMAGES AND PLOT LABELS**

In [ ]:
label_to_text = {0:'anger', 1:'disgust', 2:'sad', 3:'happiness', 4:'surprise'}

In [ ]:
emotions = [0, 1, 2, 3, 4]

for i in emotions:
    pass

In [ ]:
# @title Index Y-Axis
facial_df.emotion.value_counts().index

In [ ]:
# @title Data X-Axis
facial_df['emotion'].value_counts()

In [ ]:
# @title BarPlot
plt.figure(figsize=(10,8))
sns.barplot(y= facial_df.emotion.value_counts(),x=facial_df.emotion.unique(), color='red')
plt.title("LABELLING DATA")
plt.xlabel(label_to_text)
plt.ylabel("No.of Samples")

**Here, the sample data for *Label:1:Disgust* is very Low. The model will not be benefitted from this. Less data means low reading and less efficient model.**

## **PERFORM IMAGE AUGMENTATION & DATA  PREPERATIONt**

In [ ]:
# split the dataframe
from keras.utils import to_categorical

X = facial_df['pixels']
y = facial_df['emotion']

In [ ]:
y = to_categorical(y)

In [ ]:
y[0]

In [ ]:
X[0]

In [ ]:
# Stack : each element of X is an array of the same shape, and it stacks them along the new axis to create a new array.The resulting array X would have one additional dimension compared to the original arrays in X.
X = np.stack(X, axis=0)
X = X.reshape(24568, 96, 96, 1)

In [ ]:
print(X.shape, y.shape)

(24568, 96, 96, 1): This show, there are 24k images having 96 * 96 pixels with only 1 color channel.

In [ ]:
# Train, Test and Validation dataSet
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True) # training : 80%, test : 20%
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, shuffle=True)   # validation : 50% of test, test : 50% of test

**Shuffle:**shuffle determines whether the data should be shuffled before splitting it into training and testing sets.
* In most cases, setting shuffle to True is recommended, as it helps in producing more robust and generalizable models by preventing any biases introduced by the original order of the data.

In [ ]:
print(X_val.shape, y_val.shape)

In [ ]:
print(X_test.shape, y_test.shape)

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
(24568 - 24568 * 0.8) / 2 # validation & test data

In [ ]:
# @title image-processing : Image Normalization ranges(0-1)
X_train = X_train/255
X_val = X_val / 255
X_test = X_test / 255

**CODE:**
import numpy as np
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt
from PIL import Image

* Load an image
image = np.array(Image.open("example_image.jpg"))

* Define the cutout augmentation
augmenter = iaa.Cutout(nb_iterations=(1, 5), size=(0.1, 0.2), squared=False)

* Apply augmentation to the image
augmented_image = augmenter(image=image)

* Plot the original and augmented images
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(image)
axes[0].set_title("Original Image")
axes[0].axis("off")
axes[1].imshow(augmented_image)
axes[1].set_title("Augmented Image with Cutout")
axes[1].axis("off")
plt.show()


In [ ]:
X_train[0]

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range =15,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

## **BUILD AND TRAIN DEEP LEARNING MODEL FOR FACIAL EXPRESSION CLASSIFICATION**
 **Resnet18 or 50**
 * ResNet-18 is a shallower and lighter model, suitable for tasks where computational resources are limited or when a simpler model architecture suffices. On the other hand, ResNet-50 is deeper and more powerful, offering higher capacity and potentially better performance on complex tasks and larger datasets, albeit at the cost of increased computational requirements and training time.








In [ ]:
input_shape = (96, 96, 1)

# input tensor shape
X_input = Input(shape=input_shape)

# Zero padding
X = ZeroPadding2D((3,3))(X_input)

# 1st Stage
X = Conv2D(64, (7,7), strides = (2,2), name = 'conv1', kernel_initializer = glorot_uniform(seed = 0))(X)
X = BatchNormalization(axis = 3, name='bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D(pool_size=(3,3), strides=(2,2))(X)

# 2nd stage
X = res_block(X, filter=[64, 64, 256], stage=2)

# 3rd stage
X = res_block(X, filter=(128, 128, 512), stage=3)

# Average Pooling
X = AveragePooling2D(pool_size=(4,4), name='Average_Pooling')(X)

# Final Layer
X = Flatten()(X)
X = Dense(units=5, activation='softmax', name= 'Dense_final', kernel_initializer= glorot_uniform(seed=0))(X)
# Here, in Dense layer, we only provide 5 output as we have in the data as emotion type

model_2_emotion = Model(inputs= X_input, outputs= X, name='Resnet18')
model_2_emotion.summary()



### COMPLIE & TRAIN KEY FACIAL POINTS DETECTION deep learning model


In [ ]:
adam = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    name='adam'
  )
model_2_emotion.compile(optimizer="adam", loss="categorical_crossentropy", metrics = ['accuracy'])

**Early Stopping:**
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss',  # Monitor validation loss
                               patience=5,          # Number of epochs with no improvement after which training will be stopped
                               restore_best_weights=True)  # Restore model weights from the epoch with the best value of the monitored quantity

* monitor='val_loss': This indicates that the validation loss will be monitored for early stopping. Training will stop if the validation loss stops decreasing.

* restore_best_weights=True: This ensures that the model weights are restored to the best observed weights during training when training is stopped early. It helps prevent overfitting by reverting to the model state that performed best on the validation set.

* The .h5 extension is commonly used for saving models in HDF5 format, which is a popular file format for storing large numerical datasets. The .keras extension could be used to indicate that the file contains a model saved using Keras, but it's not a standard convention.

In [ ]:
# using early stopping to exit training if validation loss is not decreasing even after certain epochs (Patience)
earlystopping = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose=1, patience=20, restore_best_weights=True)

# save the best model with lower validation loss
checkpointer = ModelCheckpoint(filepath = "FacialExpression_weights.h5", monitor='val_loss', mode='min',  verbose=1 , save_best_only = True)

In [ ]:
history = model_2_emotion.fit(train_datagen.flow(X_train, y_train, batch_size=32), validation_data=(X_val, y_val), steps_per_epoch=len(X_train)//64, epochs= 100, callbacks=[checkpointer, earlystopping])

In [ ]:
# saving the model architecture to json file for future use
model_json = model_2_emotion.to_json()
with open("FacialExpression-model.json", "w") as json_file:
  json_file.write(model_json)

In [ ]:
# Load the model architecture from the JSON file
with open("FacialExpression-model.json", "r") as json_file:
    loaded_model_json = json_file.read()

# Reconstruct the model from the loaded architecture (Loaded_model = model_1_facialKeyPoint ho)
model_2_emotion = model_from_json(loaded_model_json)

# Load the trained weights
model_2_emotion.load_weights("weights_emotions.h5")

adam = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
# Compile the loaded model (if needed)
model_2_emotion.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

# Use the loaded model for inference or further training
# For example:
# predictions = model_1_facialKeyPoint.predict(X_test)

In [ ]:
score = model_2_emotion.evaluate(X_test, y_test)
print("Test Accuracy: ", score[1])

In [ ]:
history.history.keys()

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
epochs = range(len(accuracy))

plt.plot(epochs, accuaracy, 'bo', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()